Code taken from stackoverflow question: [Select by partial string from a pandas DataFrame](https://stackoverflow.com/questions/11350770/select-by-partial-string-from-a-pandas-dataframe)

In [11]:
import pandas as pd
import numpy as np

## Basic Substring Search

In [12]:
# setup
df1 = pd.DataFrame({'col': ['foo', 'foobar', 'bar', 'baz']})
df1

,col
0,foo
1,foobar
2,bar
3,baz


In [3]:
# find rows in `df1` which contain "foo" followed by something
df1[df1['col'].str.contains(r'foo(?!$)')]

,col
1,foobar


In [4]:
#select all rows containing "foo"
df1[df1['col'].str.contains('foo', regex=False)]
# same as df1[df1['col'].str.contains('foo')] but faster.

,col
0,foo
1,foobar


In [5]:
#Performance wise, regex search is slower than substring search:
df2 = pd.concat([df1] * 1000, ignore_index=True)

%timeit df2[df2['col'].str.contains('foo')]
%timeit df2[df2['col'].str.contains('foo', regex=False)]

2.38 ms ± 85.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.47 ms ± 260 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Addressing *ValueErrors*
Sometimes, performing a substring search and filtering on the result will result in

` ValueError: cannot index with vector containing NA / NaN values `

*This is usually because of mixed data or NaNs in your object column*

In [15]:
s = pd.Series(['foo', 'foobar', np.nan, 'bar', 'baz', 123])
s.str.contains('foo|bar')

0     True
1     True
2      NaN
3     True
4    False
5      NaN
dtype: object

*Example:*

In [16]:
s[s.str.contains('foo|bar')]

ValueError: cannot index with vector containing NA / NaN values

In [17]:
#Anything that is not a string cannot have string methods applied on it, so the result is NaN (naturally). 
#In this case, specify na=False to ignore non-string data,

s.str.contains('foo|bar', na=False)

0     True
1     True
2    False
3     True
4    False
5    False
dtype: bool

### How do I apply this to multiple columns at once?

The answer is in the question. Use `DataFrame.apply`

In [18]:
df = pd.DataFrame(
                {'A': ['foo', 'foobar', 'bar', 'baz'],
                   'B': ['barfoo', 'fuz', 'bar', 'baz']
                    }
                    )

In [19]:
# `axis=1` tells `apply` to apply the lambda function column-wise.
df.apply(lambda col: col.str.contains('foo|bar', na=False), axis=1)

,A,B
0,True,True
1,True,False
2,True,True
3,False,False


In [20]:
df = pd.DataFrame(
                {'A': ['foo', 'foobar', 'bar', 'baz'],
                   'B': ['barfoo', 'fuz', 'bar', 'baz']
                    }
                    )

In [21]:
df

,A,B
0,foo,barfoo
1,foobar,fuz
2,bar,bar
3,baz,baz


## Multiple Substring Search
This is most easily achieved through a regex search using the regex OR pipe.

In [22]:
df4 = pd.DataFrame({'col': ['foo abc', 'foobar xyz', 'bar32', 'baz 45']})
df4

,col
0,foo abc
1,foobar xyz
2,bar32
3,baz 45


In [23]:
df4[df4['col'].str.contains(r'foo|baz')]

,col
0,foo abc
1,foobar xyz
3,baz 45


You can also create a list of terms, then join them:

In [24]:
terms = ['foo', 'baz']
df4[df4['col'].str.contains('|'.join(terms))]

,col
0,foo abc
1,foobar xyz
3,baz 45


Sometimes, it is wise to escape your terms in case they have characters that can be interpreted as regex metacharacters. If your terms contain any of the following characters...
`. ^ $ * + ? { } [ ] \ | ( )`

In [25]:
import re
df4[df4['col'].str.contains('|'.join(map(re.escape, terms)))]

,col
0,foo abc
1,foobar xyz
3,baz 45


re.escape has the effect of escaping the special characters so they're treated literally.

In [26]:
re.escape(r'.foo^')

'\\.foo\\^'

### Matching Entire Word(s)

By default, the substring search searches for the specified substring/pattern regardless of whether it is full word or not. To only match full words, we will need to make use of regular expressions here—in particular, our pattern will need to specify word boundaries (\b).

For example,

In [27]:
df3 = pd.DataFrame({'col': ['the sky is blue', 'bluejay by the window']})
df3

,col
0,the sky is blue
1,bluejay by the window


Now consider,

In [28]:
df3[df3['col'].str.contains('blue')]

,col
0,the sky is blue
1,bluejay by the window


v/s

In [29]:
df3[df3['col'].str.contains(r'\bblue\b')]

,col
0,the sky is blue


## Multiple Whole Word Search

Similar to the above, except we add a word boundary (\b) to the joined pattern.

In [30]:
p = r'\b(?:{})\b'.format('|'.join(map(re.escape, terms)))
df4[df4['col'].str.contains(p)]

,col
0,foo abc
3,baz 45


In [31]:
p

'\\b(?:foo|baz)\\b'

### A Great Alternative: Use [List Comprehensions](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions)

Because you can! [And you should](https://stackoverflow.com/questions/54028199/for-loops-with-pandas-when-should-i-care)! They are usually a little bit faster than string methods, because string methods are hard to vectorise and usually have loopy implementations.

Instead of,

In [32]:
df1[df1['col'].str.contains('foo', regex=False)]

,col
0,foo
1,foobar


Use the in operator inside a list comp,

In [33]:
df1[['foo' in x for x in df1['col']]]

,col
0,foo
1,foobar


Instead of,

In [34]:
regex_pattern = r'foo(?!$)'
df1[df1['col'].str.contains(regex_pattern)]

,col
1,foobar


Use `re.compile` (to cache your regex) + `Pattern.search` inside a list comp,

In [35]:
p = re.compile(regex_pattern, flags=re.IGNORECASE)
df1[[bool(p.search(x)) for x in df1['col']]]

,col
1,foobar


### More Options for Partial String Matching:
`np.char.find, np.vectorize, DataFrame.query`

**`np.char.find`**

Supports substring searches (read: no regex) only.

In [36]:
df4[np.char.find(df4['col'].values.astype(str), 'foo') > -1]

,col
0,foo abc
1,foobar xyz


**`np.vectorize`**

This is a wrapper around a loop, but with lesser overhead than most pandas str methods.

In [37]:
f = np.vectorize(lambda haystack, needle: needle in haystack)
f(df1['col'], 'foo')
# array([ True,  True, False, False])

df1[f(df1['col'], 'foo')]

,col
0,foo
1,foobar


Regex solutions possible:

In [38]:
regex_pattern = r'foo(?!$)'
p = re.compile(regex_pattern)
f = np.vectorize(lambda x: pd.notna(x) and bool(p.search(x)))
df1[f(df1['col'])]

,col
1,foobar


**`DataFrame.query`**

Supports string methods through the python engine. This offers no visible performance benefits, but is nonetheless useful to know if you need to dynamically generate your queries.

In [39]:
df1.query('col.str.contains("foo")', engine='python')

,col
0,foo
1,foobar
